In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.0'

# A first look at a neural network

This notebook contains the code samples found in Chapter 2, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

我们来看一个具体的神经网络示例，使用 Python 的 Keras 库来学习手写数字分类。如果你 没用过 Keras 或类似的库，可能无法立刻搞懂这个例子中的全部内容。甚至你可能还没有安装 Keras。没关系，下一章会详细解释这个例子中的每个步骤。因此，如果其中某些步骤看起来有 些随意，或者像魔法一样，也请你不要担心。下面我们要开始了。

我们这里要解决的问题是，将手写数字的灰度图像（28 像素×28 像素）划分到 10 个类别 中（0~9）。我们将使用 MNIST 数据集，它是机器学习领域的一个经典数据集，其历史几乎和这 个领域一样长，而且已被人们深入研究。这个数据集包含 60 000 张训练图像和 10 000 张测试图 像，由美国国家标准与技术研究院（National Institute of Standards and Technology，即 MNIST 中 的 NIST）在 20 世纪 80 年代收集得到。你可以将“解决”MNIST 问题看作深度学习的“Hello World”，正是用它来验证你的算法是否按预期运行。当你成为机器学习从业者后，会发现 MNIST 一次又一次地出现在科学论文、博客文章等中。

The MNIST dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays:

In [2]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images 和 train_labels 组成了训练集（training set），模型将从这些数据中进行学习。然后在测试集（test set，即 test_images 和 test_labels）上对模型进行测试。

图像被编码为 Numpy 数组，而标签是数字数组，取值范围为 0~9。图像和标签一一对应。

我们来看一下训练数据：

In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
len(test_labels)

10000

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

Let's have a look at the test data:

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
len(test_labels)

10000

In [8]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

接下来的工作流程如下：

首先，将训练数据（train_images 和 train_labels）输入神经网络；

其次，网络学习将图像和标签关联在一起；

最后，网络对 test_images 生成预测， 而我们将验证这些预测与 test_labels 中的标签是否匹配。

下面我们来构建网络。再说一遍，你现在不需要理解这个例子的全部内容。

In [9]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))


神经网络的核心组件是层（layer），它是一种数据处理模块，你可以将它看成数据过滤器。 进去一些数据，出来的数据变得更加有用。具体来说，层从输入数据中提取表示——我们期望这种表示有助于解决手头的问题。大多数深度学习都是将简单的层链接起来，从而实现渐进式 的数据蒸馏（data distillation）。深度学习模型就像是数据处理的筛子，包含一系列越来越精细的数据过滤器（即层）。

本例中的网络包含 2 个 Dense 层，它们是密集连接（也叫全连接）的神经层。第二层（也 是最后一层）是一个 10 路 softmax 层，它将返回一个由 10 个概率值（总和为 1）组成的数组。每个概率值表示当前数字图像属于 10 个数字类别中某一个的概率。

要想训练网络，我们还需要选择编译（compile）步骤的三个参数。

* 损失函数（loss function）：网络如何衡量在训练数据上的性能，即网络如何朝着正确的方向前进。

* 优化器（optimizer）：基于训练数据和损失函数来更新网络的机制。

* 在训练和测试过程中需要监控的指标（metric）：本例只关心精度，即正确分类的图像所占的比例。

后续两章会详细解释损失函数和优化器的确切用途。

In [10]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


在开始训练之前，我们将对数据进行预处理，将其变换为网络要求的形状，并缩放到所 有值都在 `[0, 1]` 区间。比如，之前训练图像保存在一个 uint8 类型的数组中，其形状为 `(60000, 28, 28)`，取值区间为 `[0, 255]`。我们需要将其变换为一个 float32 数组，其形 状为 `(60000, 28 * 28)`，取值范围为 0~1。

In [11]:
train_images = train_images.reshape((60000, 28 * 28)) /255.0

test_images = test_images.reshape((10000, 28 * 28)) / 255.0

我们还需要对标签进行分类编码，第 3 章将会对这一步骤进行解释。

In [13]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels,num_classes =10)
test_labels = to_categorical(test_labels)

现在我们准备开始训练网络，在 Keras 中这一步是通过调用网络的 fit 方法来完成的— 2 我们在训练数据上拟合（fit）模型。

In [14]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 8s 135us/step - loss: 0.2558 - acc: 0.9267
Epoch 2/5
60000/60000 [==============================] - 8s 128us/step - loss: 0.1033 - acc: 0.9696
Epoch 3/5
60000/60000 [==============================] - 8s 130us/step - loss: 0.0677 - acc: 0.9796
Epoch 4/5
60000/60000 [==============================] - 8s 129us/step - loss: 0.0507 - acc: 0.9842
Epoch 5/5
60000/60000 [==============================] - 7s 123us/step - loss: 0.0374 - acc: 0.9884


训练过程中显示了两个数字：一个是网络在训练数据上的损失（loss），另一个是网络在 训练数据上的精度（acc）。 

我们很快就在训练数据上达到了 0.989（98.9%）的精度。现在我们来检查一下模型在测试 集上的性能。

In [15]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 75us/step


In [16]:
print('test_acc:', test_acc)

test_acc: 0.9804



测试集精度为 98.04%，比训练集精度低不少。训练精度和测试精度之间的这种差距是过拟合（overfit）造成的。过拟合是指机器学习模型在新数据上的性能往往比在训练数据上要差，它 是第 3 章的核心主题。

第一个例子到这里就结束了。你刚刚看到了如何构建和训练一个神经网络，用不到 20 行的 Python 代码对手写数字进行分类。下一章会详细介绍这个例子中的每一个步骤，并讲解其背后的原理。接下来你将要学到张量（输入网络的数据存储对象）、张量运算（层的组成要素）和梯度下降（可以让网络从训练样本中进行学习）。